Space X Falcon 9 First Stage Landing Prediction
Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia
Estimated time needed: 40 minutes

In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled List of Falcon 9 and Falcon Heavy launches

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches

In [1]:
!pip3 install beautifulsoup4
!pip3 install requests 

In [2]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"


TASK 1: Request the Falcon9 Launch Wiki page from its URL
First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.

In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
html_data = requests.get(static_url)
html_data.status_code

200

In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(html_data.text, 'html.parser')

In [7]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

TASK 2: Extract all column/variable names from the HTML table header
Next, we want to collect all relevant column names from the HTML table header

Let's try to find all tables on the wiki page first. If you need to refresh your memory about BeautifulSoup, please check the external reference link towards the end of this lab

In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

In [10]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

Next, we just need to iterate through the <th> elements and apply the provided extract_column_from_header() to extract column name one by one

In [11]:
column_names = []
element = soup.find_all('th')
for row in range(len(element)):
    try:
        name = extract_column_from_header(element[row])
        if (name is not None and len(name) > 0):
            column_names.append(name)
    except:
        pass
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

In [13]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'N/A', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'FH 2', 'FH 3', 'Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Date and time ( )', 'Launch site', 'Payload', '

TASK 3: Create a data frame by parsing the launch HTML tables
We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

In [14]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the launch_dict with launch records extracted from table rows.

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links B0004.1[8], missing values N/A [e], inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the launch_dict. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:

After you have fill in the parsed launch record values into launch_dict, you can create a dataframe from it.

In [17]:
# Extract the tables
extracted_row = 0

# Extract each table
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):

    # Get table row
    for rows in table.find_all("tr"):

        # (1) =================================================================
        # Check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        # (2) =================================================================
        # Get table element 
        row = rows.find_all('td')

        # (3) =================================================================
        # If it is number save cells in a dictonary 
        if flag:
            extracted_row += 1

            # (1) Flight Number value |=========| Append the flight_number into launch_dict with key `Flight No.`
            datatimelist = date_time(row[0])
            launch_dict['Flight No.']+=[flight_number]

            # (10) Date value |=========| Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date']+=[date]

            # (11) Time value |=========| Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time']+=[time]

            # (8) Booster version |=========| Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster']+=[bv]

            # (2) Launch Site |=========| Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            launch_dict['Launch site']+=[launch_site]

            # (3) Payload |=========| Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload']+=[payload]

            # (4) Payload Mass |=========| Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass']+=[payload_mass]

            # (5) Orbit |=========| Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit']+=[orbit]

            # (6) *** Customer |=========| Append the customer into launch_dict with key `Customer`
            if str(row[6])[0:7] == '<td>Var':
                print('HERE !!', row[6])
                customer = 'Various'
                print(f'Customer Name: {customer}')
            elif str(row[6])[0:26] == '<td><a href="/wiki/Turkmen':
                print('HERE !!', row[6])
                customer = 'Turkmenistan National Space Agency'
                print(f'Customer Name: {customer}')
            else:
                print('>>>Test', row[6].a.string)
                customer = row[6].a.string
            launch_dict['Customer']+=[customer]

            # (7) *** Launch outcome |=========| Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome']+=[launch_outcome]

            # (9) *** Booster landing |=========| Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing']+=[booster_landing]

>>>Test SpaceX
>>>Test NASA
>>>Test NASA
>>>Test NASA
>>>Test NASA
>>>Test MDA
>>>Test SES
>>>Test Thaicom
>>>Test NASA
>>>Test Orbcomm
>>>Test AsiaSat
>>>Test AsiaSat
>>>Test NASA
>>>Test NASA
>>>Test USAF
>>>Test ABS
>>>Test NASA
HERE !! <td><a href="/wiki/Turkmenistan_National_Space_Agency" title="Turkmenistan National Space Agency">Turkmenistan National<br/>Space Agency</a><sup class="reference" id="cite_ref-95"><a href="#cite_note-95">[88]</a></sup>
</td>
Customer Name: Turkmenistan National Space Agency
>>>Test NASA
>>>Test Orbcomm
>>>Test NASA
>>>Test SES
>>>Test NASA
>>>Test SKY Perfect JSAT Group
>>>Test Thaicom
>>>Test ABS
>>>Test NASA
>>>Test SKY Perfect JSAT Group
>>>Test Iridium Communications
>>>Test NASA
>>>Test EchoStar
>>>Test SES
>>>Test NRO
>>>Test Inmarsat
>>>Test NASA
>>>Test Bulsatcom
>>>Test Iridium Communications
>>>Test Intelsat
>>>Test NASA
>>>Test NSPO
>>>Test USAF
>>>Test Iridium Communications
>>>Test SES S.A.
>>>Test KT Corporation
>>>Test NASA
>>>Test Iri

In [18]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [20]:
# Count the number of data points in each column as a check for any errors
for count,i in enumerate(launch_dict):
    column_name = list(launch_dict.keys())[count]
    print(f'{column_name}: {length_column}')

Flight No.: 121
Launch site: 121
Payload: 121
Payload mass: 121
Orbit: 121
Customer: 121
Launch outcome: 121
N/A: 121
FH 2: 121
FH 3: 121
Demo flights: 121
logistics: 121
Crewed missions: 121
Commercial satellites: 121
Scientific satellites: 121
Military satellites: 121
Rideshares: 121
Transporter missions: 121
Current: 121
In development: 121
Retired: 121
Cancelled: 121
Spacecraft: 121
Cargo: 121
Crewed: 121
Test vehicles: 121
Unflown: 121
Orbital: 121
Atmospheric: 121
Landing sites: 121
Other facilities: 121
Support: 121
Contracts: 121
R&D programs: 121
Key people: 121
Related: 121
General: 121
People: 121
Vehicles: 121
Launches by rocket type: 121
Launches by spaceport: 121
Agencies, companies and facilities: 121
Other mission lists and timelines: 121
Version Booster: 121
Booster landing: 121
Date: 121
Time: 121


In [21]:
# Get the keys of the dictionary in a list
list(launch_dict.keys())

['Flight No.',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'N/A',
 'FH 2',
 'FH 3',
 'Demo flights',
 'logistics',
 'Crewed missions',
 'Commercial satellites',
 'Scientific satellites',
 'Military satellites',
 'Rideshares',
 'Transporter missions',
 'Current',
 'In development',
 'Retired',
 'Cancelled',
 'Spacecraft',
 'Cargo',
 'Crewed',
 'Test vehicles',
 'Unflown',
 'Orbital',
 'Atmospheric',
 'Landing sites',
 'Other facilities',
 'Support',
 'Contracts',
 'R&D programs',
 'Key people',
 'Related',
 'General',
 'People',
 'Vehicles',
 'Launches by rocket type',
 'Launches by spaceport',
 'Agencies, companies and facilities',
 'Other mission lists and timelines',
 'Version Booster',
 'Booster landing',
 'Date',
 'Time']

In [22]:
# Convert dictionary to DataFrame
df = pd.DataFrame(launch_dict)
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,N/A,FH 2,FH 3,...,People,Vehicles,Launches by rocket type,Launches by spaceport,"Agencies, companies and facilities",Other mission lists and timelines,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,None,None,None,...,None,None,None,None,None,None,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,None,None,None,...,None,None,None,None,None,None,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,None,None,None,...,None,None,None,None,None,None,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,None,None,None,...,None,None,None,None,None,None,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,None,None,None,...,None,None,None,None,None,None,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,None,None,None,...,None,None,None,None,None,None,F9 B5B1051.10,Success,9 May 2021,06:42
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,None,None,None,...,None,None,None,None,None,None,F9 B5B1058.8,Success,15 May 2021,22:56
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,None,None,None,...,None,None,None,None,None,None,F9 B5B1063.2,Success,26 May 2021,18:59
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,None,None,None,...,None,None,None,None,None,None,F9 B5B1067.1,Success,3 June 2021,17:29


In [23]:
# Export DataFrame as .csv
df.to_csv('spacex_web_scraped.csv', index=False)